## Este notebook tem como objetivo validar se as colunas das Views em um banco SQL SERVER estão de acordo com o Dicionário de Dados baseado nas colunas dos Endpoints da API do Caravel
## A função desenvolvida recebe as seguintes variáveis:
#### file_name = Localização do dicionário de dados (.xlsx);
#### sheet_name = Nome da aba da planilha onde está contida o dicionário de dados da tabela a ser validada;
#### query_sqlserver = Consulta SQL que irá retornar os metadados da tabela que será validada;
#### engine = String de conexão do banco;


## Funcionamento:
#### A função "process_dataframes", realiza um left join entre o df que contém os metadados da tabela do banco com o df do dicionário de dados através da coluna que representa o nome das colunas de ambos os dataframes("column_name"). Após o join é retornado as colunas correspondentes do dicionário de dados que são (nome de coluna, tipo de dado e se é null ou não). A partir da obtenção dessas informaçãoes é realizado a validação dos metadados. Também é apresentado as colunas que estão no dicionário de dados mas não foram encontradas na tabela do banco, seja por não existirem ou por terem nomes diferentes entre si.

## Pré-Requisitos:
#### O dicionários de dados devem ter uma tabela com as seguintes colunas: "column_name", "data_type" e "is_nullable" para realização dos join e validações.

## Regras adicionais aplicadas:
#### Durante a validação da tabela do banco, os tipos de dados (char, varchar, text, nchar, nvarchar e ntext) são consolidados como varchar.
#### Durante a validação da tabela do banco, os tipos de dados (bigint, numeric, smallint, int e numeric) são consolidados como int.
#### Obs: É possível ver essa consolidação na coluna "dtype_consolidado"

In [1]:
!pip install psycopg2-binary

In [2]:
from sqlalchemy import create_engine
import pandas as pd
from validador import DataFrameProcessor

In [3]:
#LEITURA DICIONÁRIO
file_name = "/home/jovyan/notebooks/Validação estrutura/a.xlsx"  # Certifique-se de fornecer o caminho correto do arquivo
sheet_name = 'a'

In [4]:
#LEITURA VIEW BANCO
# Configurações de conexão com o banco de dados
user = 'postgres'
password = 'password'
host = 'postgres'
port = '5432'  # Porta padrão do PostgreSQL
database = 'prova'

# Cria a string de conexão
connection_string = f'postgresql://{user}:{password}@{host}:{port}/{database}'

# Cria a engine de conexão
engine = create_engine(connection_string)

# Consulta SQL para selecionar os metadados da tabela
query_sqlserver= "SELECT column_name, data_type, is_nullable FROM information_schema.columns WHERE table_name = 'fato_avaliacoes';"

## Execução da Função

In [5]:
#processor = DataFrameProcessor(df, dic_vw)
processor = DataFrameProcessor(file_name, sheet_name, query_sqlserver, engine)
processor.read_excel_to_df()
processor.execute_sql_to_df()
processor.process_dataframes()
processor.close_session()

engine.dispose()

---------------------------------------------------------------------- RESUMO DA TABELA DO BANCO ----------------------------------------------------------------------
+--------------+-----------------------------------+----------+
|column_name   |Metadados coerentes com dicionário?|Observação|
+--------------+-----------------------------------+----------+
|id_turma      |Sim                                |OK        |
|id_disciplina |Sim                                |OK        |
|data_avaliacao|Sim                                |OK        |
|titulo        |Sim                                |OK        |
+--------------+-----------------------------------+----------+

---------------------------------------------------------------------- DETALHES DA TABELA E DICIONÁRIO ----------------------------------------------------------------------
+--------------+---------+-----------+-----------------+---------------+-------------+---------------+------------------------+------------------